<table align="left"><td><a target="_blank" href="https://colab.research.google.com/github/superannotateai/model-deployment-tutorials/blob/main/OAK/SuperAnnotate_OAK_YOLOv4_tiny_Deployment.ipynb"><img src="https://user-images.githubusercontent.com/25985824/104791629-6e618700-5769-11eb-857f-d176b37d2496.png" height="32" width="32"> Try in Google Colab</a></td></table>

# 1. Install Prerequisites 
(Please click on **RESTART RUNTIME** button when it appears in the output of this code block)



In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git
!git clone https://github.com/GotG/yolotinyv3_medmask_demo
!git clone https://github.com/TNTWEN/OpenVINO-YOLOV4.git

%cd darknet
import re
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!make
!chmod +x ./darknet
%cd ../

!sudo apt-get install -y pciutils cpio
!sudo apt autoremove
!wget http://registrationcenter-download.intel.com/akdlm/irc_nas/16803/l_openvino_toolkit_p_2020.4.287.tgz
path = "l_openvino_toolkit_p_2020.4.287.tgz"
!tar xf "{path}"

%cd l_openvino_toolkit_p_2020.4.287/
!./install_openvino_dependencies.sh && \
    sed -i 's/decline/accept/g' silent.cfg && \
    ./install.sh --silent silent.cfg

!bash /content/l_openvino_toolkit_p_2020.4.287/install_openvino_dependencies.sh
!bash /opt/intel/openvino/deployment_tools/model_optimizer/install_prerequisites/install_prerequisites.sh

!pip install superannotate
!pip install google-resumable-media==0.5.0

# 2. Setup your SuperAnnotate token, project names of trianing images and desired output model name

In [ ]:
TOKEN = "For Pro Users, the Token-ID can be generated in the teams section"
PROJECT_NAMES = ["Mask - batch 1", "Mask - batch 2"]
OUTPUT_MODEL = "sa-tutorial"

# 3. Download and preprocess the data for training

In [ ]:
import superannotate as sa
import json
import os 
from shutil import copy, make_archive
import re

proj_folder = dict(zip(PROJECT_NAMES, ["proj_" + str(i) for i in range(len(PROJECT_NAMES))]))
token_json = {"token": TOKEN}
with open('sa_config.json', 'w') as f:
  json.dump(token_json, f)

sa.init('sa_config.json')

for project_name, folder_name in proj_folder.items():
  if not os.path.exists('./training_data/' + folder_name):
    os.makedirs('training_data/' + folder_name)
  completed_images = sa.search_images(project_name, annotation_status="Completed")
  for completed_image in completed_images:
    sa.download_image(project_name, completed_image, local_dir_path='./training_data/' + folder_name)
  export = sa.prepare_export(project_name, annotation_statuses=["Completed"])
  sa.download_export(project_name, export, './training_data/' + folder_name)

class_names = set()
for folder_name in proj_folder.values():
  with open('/content/training_data/' + folder_name +'/classes/classes.json', 'r') as f:
    proj_class_data = json.load(f)
  for class_entry in proj_class_data:
    class_names.add(class_entry["name"])

class_names = list(class_names)
class_names_yolo_id = dict(zip(class_names, range(len(class_names))))

if not os.path.exists('/content/training_data/yolo_train'):
  os.mkdir('/content/training_data/yolo_train')

for folder_name in proj_folder.values():
  folder_base = '/content/training_data/' + folder_name
  folder_content = os.listdir(folder_base)
  for folder_item_path in folder_content:
    if folder_item_path.endswith("classes"):
      continue
    elif folder_item_path.endswith("json"):
      yolo_annot_file_path = '/content/training_data/yolo_train/' + folder_name + '_' + os.path.splitext(folder_item_path.split("___")[0])[0] + '.txt'
      with open(folder_base + '/' + folder_item_path, 'r') as f:
        annot_data = json.load(f)
      img_h, img_w = annot_data["metadata"]["height"], annot_data["metadata"]["width"]
      box_data = []
      for annot_inst in annot_data["instances"]:
        if annot_inst["type"] != "bbox":
          continue
        points = annot_inst["points"]
        x1, x2, y1, y2 = points["x1"], points["x2"], points["y1"], points["y2"]
        center_x = str(((x1 + x2) / 2) / img_w)
        center_y = str(((y1 + y2) / 2) / img_h)
        width = str((max(x1, x2) - min(x1, x2)) / img_w)
        height = str((max(y1, y2) - min(y1, y2)) / img_h)
        inst_class_id = str(class_names_yolo_id[annot_inst["className"]])
        box_data.append(inst_class_id + ' ' + center_x + ' ' + center_y + ' ' + width + ' ' + height + '\n')
      with open(yolo_annot_file_path, 'w') as f:
        for box_inst in box_data:
          f.write(box_inst)
    else:
      copy(folder_base + '/' + folder_item_path, '/content/training_data/yolo_train/' + folder_name + '_' + folder_item_path)

labels_path = '/content/yolotinyv3_medmask_demo/obj.names'

with open(labels_path, 'w') as f:
  for class_name, yolo_id in class_names_yolo_id.items():
    f.write(class_name + '\n')

objdata = '/content/yolotinyv3_medmask_demo/obj.data'
with open(objdata) as f:
    s = f.read()

num_classes = len(class_names_yolo_id.keys())
s = re.sub('classes= \d*','classes = ' + str(num_classes),s)

with open(objdata, 'w') as f:
  f.write(s)

%cd yolotinyv3_medmask_demo/
!python3 folder2textYolo.py 0 /content/training_data/yolo_train

# 4. Train the model

In [ ]:
max_batch= num_classes * 20 #recommended 2000
step1 = 0.8 * max_batch
step2 = 0.9 * max_batch
subdivisions = 4
num_filters = (num_classes + 5) * 3

cfg_file = '/content/yolotinyv3_medmask_demo/yolov4-tiny.cfg'

with open(cfg_file) as f:
    s = f.read()

s = re.sub('subdivisions=\d*','subdivisions='+str(subdivisions),s)
s = re.sub('max_batches = \d*','max_batches = '+str(max_batch),s)
s = re.sub('steps=\d*,\d*','steps='+"{:.0f}".format(step1)+','+"{:.0f}".format(step2),s)
s = re.sub('classes=\d*','classes='+str(num_classes),s)
s = re.sub('pad=1\nfilters=\d*','pad=1\nfilters='+"{:.0f}".format(num_filters),s)

with open(cfg_file, 'w') as f:
  f.write(s)

%cd ../darknet/

!./darknet detector train /content/yolotinyv3_medmask_demo/obj.data /content/yolotinyv3_medmask_demo/yolov4-tiny.cfg /content/yolotinyv3_medmask_demo/yolov4-tiny.conv.29 -dont_show -ext_output -map

# 5. Generate and download the files needed for the OAK-D device

In [ ]:
%tensorflow_version 1.x
from google.colab import files
import requests

files.download('/content/darknet/backup/yolov4-tiny_best.weights')

%cd /content/OpenVINO-YOLOV4/
!python3 convert_weights_pb.py --class_names /content/yolotinyv3_medmask_demo/obj.names --data_format NHWC --weights_file /content/darknet/backup/yolov4-tiny_best.weights --tiny

tiny_yolo_json = '/content/OpenVINO-YOLOV4/yolo_v4_tiny.json'
with open(tiny_yolo_json) as f:
    s = f.read()
s = re.sub('"classes": \d*','"classes": ' + str(num_classes),s)
with open(tiny_yolo_json, 'w') as f:
  f.write(s)

!source /opt/intel/openvino/bin/setupvars.sh && \
    python /opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model /content/OpenVINO-YOLOV4/frozen_darknet_yolov4_model.pb \
--transformations_config /content/OpenVINO-YOLOV4/yolo_v4_tiny.json  \
--batch 1 \
--data_type FP16 \
--reverse_input_channel \
--output_dir /content/yolotinyv3_medmask_demo/TinyIR

os.mkdir("/content/" + OUTPUT_MODEL)
blob_dir = "/content/" + OUTPUT_MODEL

binfile = "/content/yolotinyv3_medmask_demo/TinyIR/frozen_darknet_yolov4_model.bin"
xmlfile = "/content/yolotinyv3_medmask_demo/TinyIR/frozen_darknet_yolov4_model.xml"

url = "http://69.164.214.171:8083/compile"
params = {
    'version': '2020.1'
}
payload = {
    'compile_type': 'myriad', 
    'compiler_params': '-ip U8 -VPU_MYRIAD_PLATFORM VPU_MYRIAD_2480 -VPU_NUMBER_OF_SHAVES 14 -VPU_NUMBER_OF_CMX_SLICES 14'
}

model_files = {
    'definition': open(xmlfile, 'rb'),
    'weights': open(binfile, 'rb')
}
response = requests.post(url, data=payload, files=model_files, params=params)
blobname = OUTPUT_MODEL + '.blob.sh14cmx14NCE1'
with open(blob_dir + '/' + blobname, 'wb') as f:
  f.write(response.content)

%cd /content/
!wget https://raw.githubusercontent.com/luxonis/depthai/main/resources/nn/tiny-yolo-v3/model.yml
!wget https://raw.githubusercontent.com/luxonis/depthai/main/resources/nn/tiny-yolo-v3/tiny-yolo-v3.json

with open('/content/model.yml', 'r') as f:
    s = f.read()
s = re.sub('v3','v4',s)
with open('/content/' + OUTPUT_MODEL + '/model.yml', 'w') as f:
    f.write(s)
with open('tiny-yolo-v3.json', 'r') as f:
  settings_json = json.load(f)
settings_json["NN_config"]["NN_specific_metadata"]["classes"] = num_classes
settings_json["mappings"]["labels"] = list(class_names_yolo_id.keys())
with open('/content/' + OUTPUT_MODEL + '/' + OUTPUT_MODEL + '.json', 'w') as f:
  json.dump(settings_json, f)
make_archive(OUTPUT_MODEL, 'zip', OUTPUT_MODEL)
files.download(OUTPUT_MODEL + '.zip')

# 6. Copy and paste the generated model in 'depthai' directory

Please follow these steps on your local machine to get the model running

**On Windows** 

Cut the downloaded \$OUTPUT_MODEL.zip file from your Downloads folder and paste it in \$DEPTHAI_ROOT_DIR/resources/nn/ folder. Then right click on it and choose "Extract Here" option. You can delete the zip file afterwards.

**On Mac**

Cut the downloaded \$OUTPUT_MODEL.zip file from your Downloads folder and paste it in \$DEPTHAI_ROOT_DIR/resources/nn/ folder. Then double click on it. You can delete the zip file afterwards.

**On Linux**

You can either do it with GUI following the steps discussed in **On Windows** section or run the following commands in terminal

```
mv ~/Downloads/$OUTPUT_MODEL.zip $DEPTHAI_ROOT_DIR/resources/nn/
cd $DEPTHAI_ROOT_DIR/resources/nn/
unzip $OUTPUT_MODEL.zip 
rm $OUTPUT_MODEL.zip
```

# 7. Test the custom model

Afterwards you can run the new model with the following command from the $DEPTHAI_ROOT_DIR directory:

```
python3 depthai_demo.py -dd -cnn $OUTPUT_MODEL
```